In [ ]:
import os
from google.colab import drive

In [ ]:
drive.mount('/content/gdrive',force_remount=True)

Mounted at /content/gdrive


In [ ]:
Root_dir = '/content/gdrive/My Drive/transformers/'

In [ ]:
root_dir = Root_dir
os.chdir( root_dir )

print( 'Current working directory ::', os.getcwd() )

Current working directory :: /content/gdrive/My Drive/transformers


In [ ]:
#mount your drives on which code is expected to run

In [ ]:
import numpy as np


filename = 'cc.hi.300.vec'  # fastext hindi vectors https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.hi.300.vec.gz


word_vec_dim = 300 # word_vec_dim = dimension of each word vectors

def loadEmbeddings(filename):
    vocab2embd = {}

    with open(filename) as infile:
        for line in infile:
            row = line.strip().split(' ')
            word = row[0].lower()
            # print(word)
            if word not in vocab2embd:
                vec = np.asarray(row[1:], np.float32)
                if len(vec) == word_vec_dim:
                    vocab2embd[word] = vec

    print('Embedding Loaded.')
        
    return vocab2embd

# Pre-trained word embedding
vocab2embd = loadEmbeddings(filename)



vocab2embd['<UNK>'] = np.random.randn(word_vec_dim)
vocab2embd['<GO>'] = np.random.randn(word_vec_dim)
vocab2embd['<PRED>'] = np.random.randn(word_vec_dim)
vocab2embd['<EOS>'] = np.random.randn(word_vec_dim)
vocab2embd['<PAD>'] = np.zeros(word_vec_dim)

Embedding Loaded.


In [ ]:
import csv
import nltk
nltk.download('punkt')
from nltk import word_tokenize
import string

summaries = []
texts = []

def clean(text):
    text = text.lower()
    printable = set(string.printable)
    text = "".join(list(filter(lambda x: x in printable, text))) 
    return text

counter={}
max_len_text = 100
max_len_sum = 20

i=0
with open('mono.hi', 'rt') as text: #Monolingual Hindi dataset
    count=0
    for row in text:
        clean_text = word_tokenize(clean(row))
        
        if len(clean_text) <= max_len_text:
            
            for word in clean_text:
                if word in vocab2embd:
                    counter[word]=counter.get(word,0)+1
            texts.append(clean_text)
        if i%10000==0:
            print("Processing data {}".format(i))
        i+=1
        
print("Current size of data: "+str(len(texts)))


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
Processing data 0
Current size of data: 204


In [ ]:
counter

{"'": 11,
 '(': 2,
 ')': 2,
 ',': 81,
 '-': 18,
 '10': 2,
 '100': 1,
 '11': 1,
 '16': 1,
 '1971': 1,
 '1977': 1,
 '1978': 1,
 '1980': 1,
 '1985': 1,
 '1996': 1,
 '1999': 3,
 '20': 2,
 '200': 1,
 '2001': 1,
 '2003': 1,
 '2004': 1,
 '2006': 1,
 '2007': 2,
 '2010': 1,
 '2013': 1,
 '25': 1,
 '26': 1,
 '29': 1,
 '3': 1,
 '4': 1,
 '45': 1,
 '5': 1,
 '6': 1,
 '60': 1,
 '63': 1,
 '65': 1,
 '8': 1,
 '90': 1,
 '?': 7,
 'net': 1}

In [ ]:
texts

[[],
 [],
 [','],
 [','],
 [],
 [','],
 [',', ','],
 [','],
 [],
 [],
 [],
 ['2003', ','],
 [','],
 [],
 [','],
 [','],
 [','],
 [','],
 [],
 [],
 ['-'],
 [],
 [','],
 [','],
 [','],
 [','],
 [],
 [','],
 [],
 [],
 [',', '-', '-'],
 [',', ','],
 [],
 [],
 [','],
 [],
 [],
 [],
 [],
 [],
 [',', '-', ',', '-'],
 [],
 [','],
 ['45', '10', '4'],
 [',', ',', '-'],
 [',', '-', ','],
 [],
 ['-'],
 [','],
 [],
 ['?'],
 [','],
 [',', ','],
 [],
 ['25'],
 [],
 [],
 [],
 [','],
 [],
 [','],
 [',', ','],
 [],
 ['?', '?'],
 [','],
 [],
 [],
 ['6', '-'],
 ['-'],
 [',', '?', '?', '?', ','],
 [],
 [],
 [',', '2006'],
 [],
 ['3', ',', '1999'],
 [],
 ['1996'],
 ['29', ',', '2010'],
 ['20'],
 [','],
 ['20'],
 ['10'],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 ['1977'],
 ['-', '1978'],
 ['1980'],
 [],
 ['1985'],
 [],
 [],
 [],
 [],
 ['-', ','],
 [','],
 [],
 [],
 [],
 [],
 [','],
 ['2004'],
 [],
 [','],
 [],
 [','],
 [],
 [],
 [','],
 [],
 ['-'],
 [],
 [',', ','],
 ['-'],
 [','],
 [],
 [],
 [

In [ ]:
vocab = [word for word in counter]


counts = [counter[word] for word in vocab]

sorted_idx = sorted(range(len(counts)), key=counts.__getitem__)
sorted_idx.reverse()

vocab = [vocab[idx] for idx in sorted_idx]

special_tags = ["<UNK>","<GO>","<PRED>","<EOS>","<PAD>"]
if len(vocab) > 40000-len(special_tags):
    vocab = vocab[0:40000-len(special_tags)]
    

vocab += special_tags 

vocab_dict = {word:i for i,word in enumerate(vocab)}

embeddings = []
for word in vocab:
    embeddings.append(vocab2embd[word].tolist())

In [ ]:
import random

texts_idx = [idx for idx in range(0,len(texts))]
random.shuffle(texts_idx)

texts = [texts[idx] for idx in texts_idx]


In [ ]:
import random

index = random.randint(0,len(texts)-1)

print("SAMPLE CLEANED & TOKENIZED TEXT: \n\n"+str(texts[index]))


SAMPLE CLEANED & TOKENIZED TEXT: 

[',', '?']


In [ ]:
train_len = int(.7*len(texts))
val_len = int(.2*len(texts))

train_texts = texts[0:train_len]

val_texts = texts[train_len:train_len+val_len]

test_texts = texts[train_len+val_len:]

In [ ]:
def bucket_and_batch(texts, batch_size=32):
    
    global vocab_dict
    vocab2idx = vocab_dict
    
    PAD = vocab2idx['<PAD>']
    EOS = vocab2idx['<EOS>']
    UNK = vocab2idx['<UNK>']

    true_seq_lens = np.zeros((len(texts)), dtype=int)
    for i in range(len(texts)):
        true_seq_lens[i] = len(texts[i])

    sorted_by_len_indices = np.flip(np.argsort(true_seq_lens), 0)

    sorted_texts = []

    for i in range(len(texts)):
        sorted_texts.append(texts[sorted_by_len_indices[i]])

    i = 0
    batches_texts = []
    batches_true_seq_in_lens = []
    batches_true_seq_out_lens = []

    while i < len(sorted_texts):

        if i+batch_size > len(sorted_texts):
            batch_size = len(sorted_texts)-i

        batch_texts = []
        batch_true_seq_in_lens = []
        batch_true_seq_out_lens = []

        max_in_len = len(sorted_texts[i])

        for j in range(i, i + batch_size):

            text = sorted_texts[j]
            
            text = [vocab2idx.get(word,UNK) for word in text]
            
            init_in_len = len(text)

            while len(text) < max_in_len:
                text.append(PAD)
                
            batch_texts.append(text)
            batch_true_seq_in_lens.append(init_in_len)

        batches_texts.append(batch_texts)
        batches_true_seq_in_lens.append(batch_true_seq_in_lens)

        i += batch_size

    return batches_texts, batches_true_seq_in_lens

In [ ]:
train_batches_x,\
train_batches_in_lens = bucket_and_batch(train_texts)
val_batches_x,\
val_batches_in_lens = bucket_and_batch(val_texts)
test_batches_x,\
test_batches_in_lens = bucket_and_batch(test_texts)

In [ ]:
import json

diction = {}
diction['vocab']=vocab
diction['embd']=embeddings
diction['train_batches_x']=train_batches_x
diction['train_batches_in_len'] = train_batches_in_lens
diction['val_batches_x']=val_batches_x
diction['val_batches_in_len'] = val_batches_in_lens
diction['test_batches_x']=test_batches_x

diction['test_batches_in_len'] = test_batches_in_lens


with open('ProcessedData.json', 'w') as fp:
    json.dump(diction, fp)